<a href="https://colab.research.google.com/github/Trickshotblaster/nn-practices/blob/main/actual_words.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

In [2]:
#!pip install tokenizers

In [64]:
!pip install transformers
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [65]:
tokenizer.vocab_size

30522

In [4]:
#from tokenizers import Tokenizer
#from tokenizers.models import BPE
#tokenizer = Tokenizer(BPE(unk_token="[UNK]"))

In [5]:
#from tokenizers.trainers import BpeTrainer
#trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])

In [6]:
#from tokenizers.pre_tokenizers import Whitespace
#tokenizer.pre_tokenizer = Whitespace()

In [7]:
#!wget https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip
#!unzip wikitext-103-raw-v1.zip

In [8]:
#files = [f"wikitext-103-raw/wiki.{split}.raw" for split in ["test", "train", "valid"]]
#tokenizer.train(files, trainer)

In [9]:
#tokenizer.save("tokenizer-wiki.json")

In [66]:
#tokenizer = Tokenizer.from_file("tokenizer-wiki.json")

In [67]:
output = tokenizer.encode("Hello, y'all! How are you 😁 ?")
out_tensor = torch.tensor(output)

In [69]:
out_tensor

tensor([ 101, 7592, 1010, 1061, 1005, 2035,  999, 2129, 2024, 2017,  100, 1029,
         102])

In [70]:
tokenizer.decode(out_tensor)

"[CLS] hello, y'all! how are you [UNK]? [SEP]"

In [71]:
import requests
import io

# Get the content of the page
response = requests.get("https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt")

# Write the content to a file
with io.open('traintext.txt', 'w', encoding='utf-8') as f:
    f.write(response.text)

In [72]:
with open("traintext.txt", 'r') as f:
  text = f.readlines()

In [73]:
text[:10]

['First Citizen:\n',
 'Before we proceed any further, hear me speak.\n',
 '\n',
 'All:\n',
 'Speak, speak.\n',
 '\n',
 'First Citizen:\n',
 'You are all resolved rather to die than to famish?\n',
 '\n',
 'All:\n']

In [74]:
import random

In [75]:
text_len = len(text) - 1

In [104]:
context_len = 10
emb_dim = 20
#tokenizer.enable_truncation(context_len)
#tokenizer.enable_padding(direction='left', length=context_len)

In [77]:
#vocab_size = tokenizer.get_vocab_size()

pretokenize? aaaa but it takes so long

In [78]:
#text = torch.tensor([tokenizer.encode(t).ids for t in text])

In [105]:
vocab_size = tokenizer.vocab_size

In [106]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [107]:
tokenizer.pad_token_id

0

In [108]:
def make_batch(batch):
  xs = []
  ys = []
  for sentence in batch:
    context = [0] * context_len
    for word, next in zip(sentence, sentence[1:]):
      context = context[1:] + [word]
      xs.append(context)
      ys.append(next)
  xs = torch.tensor(xs).to(device)
  ys = torch.tensor(ys).to(device)
  return xs, ys

In [109]:
C = torch.randn((vocab_size, emb_dim)).to(device)
w1 = torch.randn((emb_dim * context_len, 100)).to(device) * 0.1
b1 = torch.randn(100).to(device) * 0.01
w2 = torch.randn((100, vocab_size)).to(device) * 0.1
b2 = torch.randn(vocab_size).to(device) * 0.01

params = [C, w1, b1, w2, b2]
for p in params:
  p.requires_grad = True

In [110]:
import torch.nn.functional as F

In [111]:
num_epochs = 20000
batch_size = 32
lr = 0.5

In [ ]:
for epoch in range(num_epochs):
  batch = [tokenizer.encode(text[random.randint(0, text_len)]) for i in range(batch_size)]
  #words = torch.tensor([tokenizer.encode_batch(batch)[i].ids for i in range(batch_size)])
  x, y = make_batch(batch)

  emb = C[x].view(-1, emb_dim * context_len)
  l1 = (emb @ w1 + b1).tanh()
  out = (l1 @ w2 + b2).tanh()
  loss = F.cross_entropy(out, y)

  for p in params:
    p.grad = None

  loss.backward()
  for p in params:
    p.data -= p.grad * lr
  if epoch % 1000 == 0:
    print(loss.item())

10.473085403442383
9.286877632141113
8.91584587097168
8.8970308303833
8.869790077209473
8.874558448791504
8.785171508789062
8.769054412841797
8.781452178955078
8.78555679321289
8.806958198547363
8.658671379089355
8.792817115783691
8.789236068725586
8.800768852233887


In [97]:
loss.log10().item()

0.9405363202095032

In [98]:
@torch.no_grad()
def forward(text):
  ins = text
  x = tokenizer.encode(text)
  context = [0] * (context_len - len(x)) + x if (context_len - len(x)) > 0 else x[:-context_len]
  context = torch.tensor(context)
  emb = C[context].view(-1, emb_dim * context_len)
  l1 = (emb @ w1 + b1).tanh()
  out = (l1 @ w2 + b2).tanh()
  out = out.exp()
  out /= out.sum(1, keepdim=True)
  sampled = torch.multinomial(out, num_samples=1)
  return sampled

In [99]:
def numerical_forward(cont):
  cont = torch.tensor(cont).to(device)
  emb = C[cont].view(-1, emb_dim * context_len)
  l1 = (emb @ w1 + b1).tanh()
  out = (l1 @ w2 + b2).tanh()
  out = out.exp()
  out /= out.sum(1, keepdim=True)
  sampled = torch.multinomial(out, num_samples=1)
  return sampled

In [100]:
lol = forward('Romeo')
print(lol)
tokenizer.decode(lol[0][0])

tensor([[10135]], device='cuda:0')


'carpet'

ok this is actually crazy it said thou

In [101]:
tokenizer.vocab_size

30522

In [102]:
def cont(text, max_len=20):
  x = tokenizer.encode(text)
  context = [0] * (context_len - len(x)) + x if (context_len - len(x)) > 0 else x[:-context_len]

  for x in range(max_len):

    context.append(numerical_forward(context).cpu().numpy()[0][0])
    context = context[1:]

  return tokenizer.decode(context)

cont("Romeo")

'thin threw ɡ humid jungle georgia islamic vengeance selecting location mouse invoked mediationdron linluvaryium rests blamed'

In [93]:
tokenizer.encode("[UNK]")

[101, 100, 102]

In [94]:
tokenizer.vocab_size

30522